<a href="https://colab.research.google.com/github/DonRoboto/AprendizajeAutomatizado/blob/master/BayesianNetwork_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install pomegranate


In [50]:

import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import math
from pomegranate import *


In [157]:
df = pd.read_csv("audit_risk.csv") 

df.head()

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,23,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,1.0,3.38,0.2,0.676,2,0.2,0.4,0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1
1,3.89,6,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5.0,0.2,1.0,0.94,0.2,0.188,2,0.2,0.4,0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0
2,3.89,6,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0
3,3.89,6,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,3.6,11.75,0.6,7.050,2,0.2,0.4,0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1
4,3.89,6,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.416,0.4,0.5,0.2832,0


In [170]:
def jitter(a_series, noise_reduction=1000000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))

#Sector_score 	LOCATION_ID 	PARA_A 	Score_A 	
#Risk_A 	PARA_B 	Score_B 	Risk_B 	TOTAL 	numbers
# 	Score_B.1 	Risk_C 	Money_Value 	Score_MV 	Risk_D 	
#   District_Loss 	PROB 	RiSk_E 	History 	Prob 	Risk_F 	
#   Score 	Inherent_Risk 	CONTROL_RISK 	Detection_Risk 	Audit_Risk

 	 	
partitions = 10

df['D_Sector_score'] = pd.qcut(df['Sector_score']+ jitter(df['Sector_score']), partitions, labels=False)
df['D_PARA_A'] = pd.qcut(df['PARA_A']+ jitter(df['PARA_A']), partitions, labels=False)
df['D_Risk_A'] = pd.qcut(df['Risk_A']+ jitter(df['Risk_A']), partitions, labels=False)
df['D_Score_A'] = pd.qcut(df['Score_A']+ jitter(df['Score_A']), partitions, labels=False)

df['D_PARA_B'] = pd.qcut(df['PARA_B']+ jitter(df['PARA_B']), partitions, labels=False)
df['D_Score_B'] = pd.qcut(df['Score_B']+ jitter(df['Score_B']), partitions, labels=False)
df['D_Risk_B'] = pd.qcut(df['Risk_B']+ jitter(df['Risk_B']), partitions, labels=False)




In [159]:
df.head()


,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk,D_Sector_score,D_Risk_A,D_Score_A,D_PARA_A
0,3.89,23,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5.0,0.2,1.0,3.38,0.2,0.676,2,0.2,0.4,0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1,12,16,17,16
1,3.89,6,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5.0,0.2,1.0,0.94,0.2,0.188,2,0.2,0.4,0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0,10,3,7,2
2,3.89,6,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0,11,7,9,7
3,3.89,6,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,3.6,11.75,0.6,7.050,2,0.2,0.4,0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1,12,1,2,1
4,3.89,6,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,1.0,0.00,0.2,0.000,2,0.2,0.4,0,0.2,0.0,2.0,1.416,0.4,0.5,0.2832,0,9,3,7,3


In [174]:
df_model = df.loc[:,['Risk','D_Sector_score', 'D_PARA_A', 'D_Risk_A', 'D_Score_A', 'D_PARA_B', 'D_Score_B', 'D_Risk_B']]

In [175]:
df_model.head()

,Risk,D_Sector_score,D_PARA_A,D_Risk_A,D_Score_A,D_PARA_B,D_Score_B,D_Risk_B
0,1,12,16,16,18,14,4,13
1,0,11,2,2,5,15,2,14
2,0,11,7,7,8,9,3,9
3,1,10,1,2,10,16,19,16
4,0,10,3,1,9,8,6,8


In [176]:
X = df_model.loc[:,['D_Sector_score', 'D_PARA_A', 'D_Risk_A', 'D_Score_A', 'D_PARA_B', 'D_Score_B', 'D_Risk_B']]
y = df_model.loc[:,['Risk']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train_model = pd.concat([y_train, X_train], axis=1)

model = BayesianNetwork.from_samples(X_train_model, state_names=X_train_model.columns.values, algorithm='exact', pseudocount=1.7)


In [ ]:
X_test_model = pd.concat([y_test, X_test], axis=1)
X_test_model['Risk']=None


In [98]:
#X_test_model.values


In [121]:
#predict = model.predict_proba(X_test_model.values)

In [ ]:
y_model = model.predict(X_test_model.values)

#y_model

In [ ]:
def Extract(lst):
    return list(list(zip(*lst))[0])

y_pred = Extract(y_model)

y_true = y_test

In [ ]:
cf = confusion_matrix(y_true, y_pred)
cf